# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [99]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

data = pd.read_csv('/datasets/data.csv') #считываем таблицу
data.info() #получаем общую информацию по таблице
print(data.head(20)) #получаем 10 первых значений 
print(data.tail(20)) #получаем 10 последних значений 
print('Уникальные значения в солбце children:')
print(data['children'].unique())
print('\nУникальные значения в education:')
print(data['education'].unique())
print(data['education_id'].unique())
print('\nУникальные значения в family_status:')
print(data['family_status'].unique())
print(data['family_status_id'].unique())
print('\nУникальные значения в gender:')
print(data['gender'].unique())
print('\nУникальные значения в income_type:')
print(data['income_type'].unique())
print('\nУникальные значения в debt:')
print(data['debt'].unique())
print('\nУникальные значения в dob_years:')
print(data['dob_years'].sort_values().unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124

**Вывод**

- children(отрицательное кол-во детей, 20 детей многовато)
- days_employed(NaN значения, тип Float, отрицательные значения, нереальные значения)
- dob_years(нулевые значения)
- education(регистр)
- total_income(Nan значения)
- purpose(лемматизация значений для объединения в категории)

In [100]:
print(data['children'].value_counts()) #кол-во строк с отрицательным значением в стобце "children"
data['children']=data['children'].replace(-1,1) #заменяем -1 на 1, скорее всего ошибка ввода
data['children']=data['children'].replace(20,2) #заменяем 20 на 2, скорее всего ошибка ввода(слишком много значений)
print(data['children'].value_counts()) #проверка

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


In [101]:
data['education'] = data['education'].str.lower() #перевод строк в нижний регистр
print(data['education'].value_counts()) #проверка

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


In [102]:
print(data['days_employed'].describe()) #посмотрим информацию по столбцу "общий трудовой стаж в днях"
data['days_employed'] = data['days_employed'].abs() #переведем отрицательные значения в положительные
print('------------------------------')
print(data['days_employed'].describe()) #среднее значение зашкаливает за 165 лет

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64
------------------------------
count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64


## Шаг 2. Предобработка данных

### Обработка пропусков

In [103]:
print('Данные по пропускам:\n',data.isna().sum()) #вывод данных по суммарному количеству пропусков
income_mean = data['total_income'].mean() #среднеее значение по столбцу ежемесячный доход
print('\nСредний ежемесячный доход:',income_mean)
days_employed_mean = data['days_employed'].mean() #среднеее значение по столбцу общий трудовой стаж в днях
print('Средний общий трудовой стаж в днях:',days_employed_mean)
print(data[data['total_income'].isna()]['income_type'].value_counts())#рассмотрим по типу занятости
data['total_income'] =data['total_income'].fillna(income_mean)#замена пропущенных значений на среднее значение
data['days_employed'] = data['days_employed'].fillna(0)#замена пропущенных значений на 0
print('\n')
data.info()#проверка после замены

Данные по пропускам:
 children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Средний ежемесячный доход: 167422.30220817294
Средний общий трудовой стаж в днях: 66914.72890682236
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender    

**Вывод**

Количество пропусков в столбцах "общий трудовой стаж в днях" и "ежемесячный доход", скорее всего это записи о людях, которые устроены не официально, либо скрывают данные, либо ошибки при выгрузке.
Посмотрим на среднее значение, среднее в столбце "общий трудовой стаж в днях" - не реальное,а вот "ежемесячных доход" выглядит нормально.

Удалять строки с пропусками в столбцах total_income и days_employed не целесобразно, присутсвует информация, которая понадобится. По этому, Nan-значения days_employed на ноль, а total_income на среднее значение.


### Замена типа данных

In [104]:
data['days_employed'] = data['days_employed'].astype('int64') # перевод значений в целочисленый тип
data['total_income'] = data['total_income'].astype('int64') # перевод значений в целочисленый тип
data.info() # проверка данных о таблице
data.head(20) #вывод 20 строк для точной проверки

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

Перевод значений из float64 в int64, в столбцах(ежемесячный доход и общий трудовой стаж в днях) для лучшего восприятия. Что делать с отрицательным и нереальным стажем не понятно...

### Обработка дубликатов

In [105]:
print ('Дубликатов:',data.duplicated().sum()) #поиск кол-ва дубликатов
data = data.drop_duplicates().reset_index(drop=True) #удаляем дубликаты
print ('После удаления:',data.duplicated().sum()) #проверка

Дубликатов: 71
После удаления: 0


**Вывод**

Удлили дубликаты с изменением индексов.

### Лемматизация

In [106]:
            
unique_purposes = data['purpose'].unique()#смотрим уникальные значения в столбце "purpose"
print('Уникальные значения в столбце "цель кредита":',unique_purposes)
lemmas_string = ' '.join(unique_purposes)#создаем строку из списка unique  с разделителем
lemmas = m.lemmatize(lemmas_string)
new_lemmas = [] #создаем новый список без мусора
for i in range(len(lemmas)):
    if len(lemmas[i])>3:
        new_lemmas.append(lemmas[i])#если длинна элемента больше 3, забираем 
print('Список без лишних элементов:')
print(Counter(new_lemmas))
purpose_groups = ['недвижимость', 'автомобиль', 'образование', 'свадьба']

Уникальные значения в столбце "цель кредита": ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высши

**Вывод**

Выделить можно четыре основные категории: недвижимость, автомобиль, образование и свадьба. Сохранили в список, для разделения запросов по категориям.

### Категоризация данных

In [107]:
data['total_income'].quantile([0.25,0.5,0.75])#выделим квартили для разбиения дааный о доходах на категории

0.25    107623.00
0.50    156043.50
0.75    195813.25
Name: total_income, dtype: float64

In [108]:
def purpose_group(purpose): #функция возвращает категорию цели кредита, принимает аргумент: цель кредита
    lem = m.lemmatize(purpose) #лемматизация строки с русским текстом
    for i in range(len(purpose_groups)):
        if purpose_groups[i] in lem:
            return(purpose_groups[i])
        elif 'жилье' in lem: #категории "жилье" нет в нашем списке,рассматриваем отдельно и в случае совпадения возвращаем категорию "недвижимость"
            return 'недвижимость'
    return('Ошибка классификации по запросу:',purpose)

def total_income_group(total_income):#функия возвращает категорию дохода, принимаемы аргументы: доход
    if 0 <= total_income <= 89088.50:
        return 'низкий'
    elif 135781.00 >= total_income > 89088.50:
        return 'средний'
    elif 195813.25>=total_income >= 135781.00:
        return 'высокий'
    elif total_income > 195813.25:
        return 'сверхвысокий'
    else:
        return('Ошибка классификации по доходу населения:',total_income)
    
data['purpose_group'] = data['purpose'].apply(purpose_group)
data['total_income_group'] = data['total_income'].apply(total_income_group)
print(data['purpose_group'].value_counts())
print(data['total_income_group'].value_counts())

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_group, dtype: int64
высокий         7466
сверхвысокий    5364
средний         5363
низкий          3261
Name: total_income_group, dtype: int64


**Вывод**

По категориям целей, можно сделать вывод о том,что на недвижимость берут чаще кредит,чем на другие цели.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [110]:
children_tab = data.pivot_table(index=['children'],columns='debt', values = 'total_income',aggfunc='count')#формируем сводную таблицу
children_tab = children_tab.fillna(0) #заменим значение NaN на 0, людей с просрочкой с 5 детьми нет
children_return = data[(data['children']>0) & (data['debt']==0)]['debt'].count()/data[data['children']>0]['debt'].count()#процент возврата с детьми
not_children_return = data[(data['children']==0) & (data['debt']==0)]['debt'].count()/data[data['children']==0]['debt'].count()#процент возврата без детей
children_tab['repayment'] = children_tab[0]/(children_tab[0]+children_tab[1]) #добавляем столбец с % возврата по каждой позиции
print(children_tab)
if children_return > not_children_return:
    print('Люди с детьми имеют меньше просрочек,отдают в срок:{:.1%}'.format(children_return))
elif children_return < not_children_return:
    print('Люди без детей имеют меньше просрочек,отдают в срок:{:.1%}'.format(not_children_return))
elif children_return == not_children_return:
    print('Зависимости между наличием детей и возвратом кредита в срок нет, % возврата обоих категорий:{:.1%}'.format(not_children_return))   

debt            0       1  repayment
children                            
0         13028.0  1063.0   0.924562
1          4410.0   445.0   0.908342
2          1926.0   202.0   0.905075
3           303.0    27.0   0.918182
4            37.0     4.0   0.902439
5             9.0     0.0   1.000000
Люди без детей имеют меньше просрочек,отдают в срок:92.5%


**Вывод**

Люди без детей имеют меньше просрочек, связано это с тем,что дети несут с собой дополнительную финансовую нагрузку на семью. Так же велик риск непредвиденных расходов. Даже не смотря на мат.капиталы платежеспособность семей не так высока,как без детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [111]:
family_tab = data.pivot_table(index=['family_status'],columns='debt',values='dob_years',aggfunc='count')
family_tab['repayment'] = family_tab[0]/(family_tab[0]+family_tab[1])
print(family_tab.sort_values(by='repayment'))

debt                       0    1  repayment
family_status                               
Не женат / не замужем   2536  274   0.902491
гражданский брак        3763  388   0.906529
женат / замужем        11408  931   0.924548
в разводе               1110   85   0.928870
вдовец / вдова           896   63   0.934307


In [112]:
df_example = data.pivot_table(index = 'children', values = 'debt', 
                            aggfunc = ['count', 'sum', 'mean', lambda x: 1 - x.mean()])
df_example.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
df_example.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

,Кол-во пользователей,Кол-во должников,% должников,% НЕдолжников
children,,,,
0,14091,1063,7.54%,92.46%
1,4855,445,9.17%,90.83%
2,2128,202,9.49%,90.51%
3,330,27,8.18%,91.82%
4,41,4,9.76%,90.24%
5,9,0,0.00%,100.00%


**Вывод**

Самые низкий процент у категорий: "Не женат / не замужем" и "гражданский брак". Больше всего кредиты берут люди в браке и отдают их вовремя, самый рентабельный вариант.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [113]:
income_tab = data.pivot_table(index=['total_income_group'],columns='debt',values='dob_years',aggfunc='count')
income_tab['repayment'] = income_tab[0]/(income_tab[0]+income_tab[1])
print(income_tab.sort_values(by='repayment'))

debt                   0    1  repayment
total_income_group                      
высокий             6815  651   0.912805
средний             4905  458   0.914600
низкий              3012  249   0.923643
сверхвысокий        4981  383   0.928598


**Вывод**

% возврата кредита в срок самый низкий у категорий близких к медиане. Но разница между показателями не сильно велика, в пределах 2%,чтоб делать какой то вывод, скорее нет никакой зависимости.

- Как разные цели кредита влияют на его возврат в срок?

In [114]:
purpose_tab = data.pivot_table(index=['purpose_group'],columns='debt',values='dob_years',aggfunc='count')
purpose_tab['repayment'] = purpose_tab[0]/(purpose_tab[0]+purpose_tab[1])
print(purpose_tab.sort_values(by='repayment'))

debt               0    1  repayment
purpose_group                       
автомобиль      3903  403   0.906410
образование     3643  370   0.907800
свадьба         2138  186   0.919966
недвижимость   10029  782   0.927666


**Вывод**

Самые рентабельные виды кредитования в недвижимость. Самые не рентабельные - авто кредиты и на образование. Свадьбы удивили...

## Шаг 4. Общий вывод

Самые надежные направление: ипотечное кредитование семьям, без детей.

Стоит обратить внимание заказчику на данные, которые они прислали:
- Нереальный трудовой стаж, не понятно как с эти работать
- Нет уникальных индефикаторов(например, номер кредитного договора)

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.

P.S.: Блок try-except использовать, по моему мнению, негде.